<a href="https://colab.research.google.com/github/anthony-illenden/mea705/blob/main/week2_advection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Figure Size (Default is too small)
plt.rcParams["figure.figsize"] = (15, 9)

**Compute advection in 1 dimension**

Can desingate a source and/or an initial distribution of a tracer

In [ ]:
# Define spatial grid
x_domain = 2000. #domain width: km
x_max = x_domain*1.e3 #domain width: m
n_x=2001 #number of points
delta_x = x_max/(n_x-1) #grid spacing: m
x=np.linspace(0.,x_max,n_x) #x values: m
x_km=x/1000.

# Horizontal wind, U
U = np.empty(n_x)
U_const = 20.
U[:] = U_const #simplest case is uniform wind: m/s

# Define times
t_duration = 72 #duration of run: hours
t_max = t_duration*3600 #duration of run: s
delta_t = .5*delta_x/U_const #timestep to satisfy CFL criteria
n_t=int(t_max/delta_t) #number of timesteps
t=np.linspace(0.,t_max,n_t) #time values: s
t_hours=t/3600.

# Tracer array in x & t
Tracer=np.zeros((n_x,n_t))

# Tracer parameters: dissipation rate and source
Decay_rate = 2.e-5# tracer decay rate:  1/s

Source = np.zeros(n_x) #distribution of tracer source: conc/s

#Put source region near western boundary
Source_mag= 1.e-5 #tracer units/s
Source_center=200. # location of source: km
Source_width=300. # width of source region: km
Source_w = Source_center - Source_width/2.
Source_e = Source_center + Source_width/2.
for i_x in range(0,n_x):
  if x_km[i_x] >= Source_w and x_km[i_x] <= Source_e:
    Source[i_x]=.5*Source_mag*(1.-np.cos((x[i_x]-Source_w)*2.*np.pi/Source_width))

# Initial tracer profile
Tracer_mag=0. #magnitude of tracer blob: tracer units
Tracer_width=600.e3 #width of blob: m
Tracer_center=350.e3 #center of blob: m
Tracer_e = Tracer_center+Tracer_width/2.
Tracer_w = Tracer_center-Tracer_width/2.
for i_x in range(0,n_x):
  if x[i_x] >= Tracer_w and x[i_x] <= Tracer_e:
    Tracer[i_x,0]=.5*Tracer_mag*(1.-np.cos((x[i_x]-Tracer_w)*2.*np.pi/Tracer_width))

# Loop over time using explicit upstream scheme
# Split advection, source, and sink
for i_t in range(1,n_t):
  Tracer[:,i_t]=Tracer[:,i_t-1]

  #Advection
  for i_x in range (1,n_x):
    U_minus =.5*(U[i_x-1]+U[i_x])
    if U_minus > 0:
      Tracer[i_x,i_t] = Tracer[i_x,i_t]\
       + .5*delta_t*U_minus*(Tracer[i_x-1,i_t]-Tracer[i_x,i_t])/delta_x
  for i_x in range (0,n_x-1):
    U_plus = .5*(U[i_x+1]+U[i_x])
    if U_plus < 0:
      Tracer[i_x,i_t] = Tracer[i_x,i_t]\
       + .5*delta_t*U_plus*(Tracer[i_x,i_t]-Tracer[i_x+1,i_t])/delta_x

  #Source & decay
  Tracer[:,i_t]=(Tracer[:,i_t]+delta_t*Source)/(1.+delta_t*Decay_rate)


Plot initial & final tracer distributions

In [ ]:
#Plots
#Initial and final tracer distributions
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.set_ylabel('Tracer')
ax1.set_xlabel('x_km (km)')
ax1.set_title('Initial tracer distribution')
line, = ax1.plot(x_km,Tracer[:,0])
plt.show()
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.set_ylabel('Tracer')
ax1.set_xlabel('x_km (km)')
ax1.set_title('Final tracer distribution')
line, = ax1.plot(x_km,Tracer[:,n_t-1])

Contour plot of tracer distribution over time

In [ ]:
# Contour plot of tracer over time
fig, ax = plt.subplots()
Tracer_trans=np.matrix.transpose(Tracer)
CS = ax.contourf(x_km,t_hours, Tracer_trans)
cbar = plt.colorbar(CS)
ax.set_title('Tracer over time');
ax.set_xlabel('x_km (km)');
ax.set_ylabel('t (hours)');